### 웹 스크래이핑
* 개요
    - 웹의 정보를 활용하여 데이터 생성이 목적
    - 웹의 원시 코드(원본)을 읽어들여 해당하는 데이터 추출
* 라이브러리
    - request : 데이터 원본 코드 가져오기
    - BeautifulSoup : html->DOM 객체기반의 요소 검색에 최적화
    - sqlite : 경량 파일기반 데이터 베이스
    

In [1]:
# url =https://finance.naver.com/item/sise.naver?code=003490
import requests
#pip install beautifulsoup4
from bs4 import BeautifulSoup as bsp
import sqlite3

In [2]:
url='https://finance.naver.com/item/sise.naver?code=003490'
# url 에 접속해서 데이터 가져오기
src=requests.get(url)
# 데이터의 content 를 가져오기
html=bsp(src.text,'html.parser')
# css 셀렉터 기반의 서치 (태그 클래스 아이디)
htitle=html.select('title')
htitle[0].getText()
hcode=html.select('.code')
hcode[0].getText()


'003490'

In [12]:
# 허들 1: 데이터 원본상에 해당 요소를 찾을수 없을때
# 소스 원본상에 원하는 데이터를 찾을 수 없어서 내부 분석결과  
# iframe 태그발견  src 가져옴
newurl='https://finance.naver.com/item/sise_day.naver?code=003490'
src=requests.get(newurl)
src.text[-600:]
# iframe 소스를 긁어와도 찾을수 없음으로 나올때

'esc2">방문하시려는 페이지의 주소가 잘못 입력되었거나,<br>\n\t\t페이지의 주소가 변경 혹은 삭제되어 요청하신 페이지를 찾을 수 없습니다.<br>\n\t\t입력하신 주소가 정확한지 다시 한번 확인해 주시기 바랍니다.\n\t</p>\n\t<p class="error_desc3">관련 문의사항은 <a href="https://help.naver.com/" target="_blank">고객센터</a>에 알려주시면 친절히 안내해드리겠습니다. 감사합니다.</p>\n\t<p class="btn">\n\t\t<a href="javascript:history.back()"><img src="https://ssl.pstatic.net/static/common/error/090610/btn_prevpage.gif" width="115" height="35" alt="이전 페이지로"></a>\n\t\t<a href="https://finance.naver.com"><img src="https://ssl.pstatic.net/static/nfinance/btn_home.gif" width="115" height="35" alt="금융홈으로"></a>\n\t</p>\n</div>\n</body>\n</html>\n'

### 해결1: 서버 속이기
* 헤더 정보를 브라우저라고 속이기
    - headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'}

#### 해결2: 직접 브라우저 조작하기
* webdriver 이용하기

In [22]:
# 허들2:  헤더정보를 속여서 브라우저인것 처럼 알림
# 헤더정보를 속이기
h = {'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'}
newurl='https://finance.naver.com/item/sise_day.naver?code=003490'
src=requests.get(newurl,headers=h)
#src.text[:600]
htdoc=bsp(src.text,'html.parser')
# css 셀렉터
tgt=htdoc.select('.red03')
if(tgt):
    print(tgt)
else:
    print('데이터가 존재하지 않습니다.')

[<span class="red03">일별</span>]


In [68]:
# 데이터 베이스 사용하기 SQLITE3
def insdata(d=[]):
    dbname='../_db/stockdb.db'
    if(len(d)==9):
        with sqlite3.connect(dbname) as conn:
            cur=conn.cursor()
            sql="insert into stockp (skey,scode,sname,sday,sopenp,shighp,slowp,sclosep,svol) values(?,?,?,?,?,?,?,?,?)"
            cur.execute(sql,d)
    else:
        print('입력데이터 오류')
    print('.',end='')


In [77]:
def getPage(no='1'):
    h = {'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'}
    newurl='https://finance.naver.com/item/sise_day.naver?code=003490&page='+no
    print(newurl)
    src=requests.get(newurl,headers=h)
    htdoc=bsp(src.text,'html.parser') 
    tbl=htdoc.select('table.type2 tr')
    # 데이터가 있는 라인만 접근
    for tr in tbl:
        tday=tr.select('.p10')
        if(len(tday)):
            tds=tr.select('td')
            sday=tds[0].getText().strip().replace('.','-')
            sendp=tds[1].getText().strip().replace(',','')
            sstartp=tds[3].getText().strip().replace(',','')
            shighp=tds[4].getText().strip().replace(',','')
            slowp=tds[5].getText().strip().replace(',','')
            svol=tds[6].getText().strip().replace(',','')
            sktxt=sday.replace('-','')+'003490'
            skey=int(sktxt)
            d=[skey,'003490','대한항공',sday,sendp,sstartp,shighp,slowp,svol]
            insdata(d)


In [79]:
for i in range(1,2):
    getPage(str(i))

https://finance.naver.com/item/sise_day.naver?code=003490&page=600


IntegrityError: UNIQUE constraint failed: stockp.skey

In [ ]:
###
